In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


import pandas as pd
from pathlib import Path
import dateutil.parser
from NPw_aux import read_iono_data, process_ionosphere_files, read_EQ_data, process_eq
from datetime import timedelta

In [2]:
p = Path("/home/carloscano/data/EQ/").glob("*.csv")
all_pd = pd.DataFrame()

valid_colums = ["time", "longitude", "latitude", "depth", "mag", "type"]
for file in p:
    print(file)
    all_pd = pd.concat([all_pd, pd.read_csv(file,  parse_dates=['time'], date_parser= dateutil.parser.parse).loc[:, valid_colums]])
all_pd = all_pd[all_pd['type'] == "earthquake"].drop("type", axis = 1).set_index("time").sort_index()
all_pd.to_csv("2016_2020_EQ.csv")

/home/carloscano/data/EQ/2017_01_06.csv
/home/carloscano/data/EQ/2018_07_09.csv
/home/carloscano/data/EQ/2018_04_06.csv
/home/carloscano/data/EQ/2020_07_12.csv
/home/carloscano/data/EQ/2019_01_06.csv
/home/carloscano/data/EQ/2019_07_12.csv
/home/carloscano/data/EQ/2020_01_06.csv
/home/carloscano/data/EQ/2018_01_03.csv
/home/carloscano/data/EQ/2017_07_12.csv
/home/carloscano/data/EQ/2016_01_06.csv
/home/carloscano/data/EQ/2018_09_12.csv
/home/carloscano/data/EQ/2016_07_12.csv


In [3]:

df_eq = read_EQ_data("/home/carloscano/data/EQ/")

/home/carloscano/data/EQ/2017_01_06.csv
/home/carloscano/data/EQ/2018_07_09.csv
/home/carloscano/data/EQ/2018_04_06.csv
/home/carloscano/data/EQ/2020_07_12.csv
/home/carloscano/data/EQ/2019_01_06.csv
/home/carloscano/data/EQ/2019_07_12.csv
/home/carloscano/data/EQ/2020_01_06.csv
/home/carloscano/data/EQ/2018_01_03.csv
/home/carloscano/data/EQ/2017_07_12.csv
/home/carloscano/data/EQ/2016_01_06.csv
/home/carloscano/data/EQ/2018_09_12.csv
/home/carloscano/data/EQ/2016_07_12.csv


In [4]:
df_process = process_eq(df_eq,df_ion.columns, 5, 2000, 1000, 4.5, 1.5)

NameError: name 'df_ion' is not defined

In [ ]:
len(df_process)

In [ ]:
df_process

In [ ]:
from NPw_aux import prepare_ion_data

In [ ]:
df_ion, df_eq = prepare_ion_data("GRK")

In [76]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from aux_function_SR import read_data, get_eq_filtered, SR_SENSORS
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_log_level, save, load
import logging
import pandas as pd
from datetime import datetime, timedelta
import pickle
from NPw import NPw, ConfigEQ, ConfigNPw, ConfigForecast
from dataclasses import dataclass, asdict
from dateutil.relativedelta import *
from NPw_aux import prepare_ion_data, process_ionosphere_files, remove_outliers
import ast

import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
set_log_level("ERROR")

delta = timedelta(minutes=30)
# Read SR and EQ data
freq = timedelta(minutes = 60)
df_ion, df_eq = prepare_ion_data("GRK", freq)
remove_outliers(df_ion, 3)

/home/carloscano/data/EQ/
/home/carloscano/data/EQ/2017_01_06.csv
/home/carloscano/data/EQ/2018_07_09.csv
/home/carloscano/data/EQ/2018_04_06.csv
/home/carloscano/data/EQ/2020_07_12.csv
/home/carloscano/data/EQ/2019_01_06.csv
/home/carloscano/data/EQ/2019_07_12.csv
/home/carloscano/data/EQ/2020_01_06.csv
/home/carloscano/data/EQ/2018_01_03.csv
/home/carloscano/data/EQ/2017_07_12.csv
/home/carloscano/data/EQ/2016_01_06.csv
/home/carloscano/data/EQ/2018_09_12.csv
/home/carloscano/data/EQ/2016_07_12.csv


In [92]:
# Create the prior dataframe
df_regressor = df_ion.reset_index().copy()
df_events = df_eq.copy()
from NPw import NPw, ConfigEQ, ConfigNPw, ConfigForecast

forecast_length = timedelta(hours=48)
question_mark_length = timedelta(hours=48)
# Time to take into account to predict 
historic_lenght =  timedelta(days=5)
training_lenght = timedelta(days=365)
config_npw_d = {
    "forecast_length": forecast_length,
    "freq": freq,
    "question_mark_length": question_mark_length,
    "num_hidden_layers": 10,
    "learning_rate": None,
    "d_hidden": 16,
    "verbose": True,
    "epochs": None,
    "gpu": False,
    "binary_event": False,
    "drop_missing": False,
    "historic_lenght": historic_lenght,
    "multivariate_global": True,
    "training_length": training_lenght,
    "normalization": True,
    "regularization": 0.4
}
config_npw = ConfigNPw(**config_npw_d)
ConfigEQ_d = {
    "dist_start": 500,
    "dist_delta": 3000,
    "mag_start": 4.5,
    "mag_delta": 2,
    "filter": True,
}
config_events = ConfigEQ(**ConfigEQ_d)
#config_path = "config_002.json"
#(config_npw, config_events) = NPw.save_config(config_path)

In [93]:
NPw_o = NPw(config_npw, df_regressor, config_events, df_events)

m1: 4.5 m2: 6.5 d1: 500 d2: 3500


In [94]:
hours_offsets = [0]
event_offsets = [None,  -timedelta(hours=24)]
index = 1


# see what happen in 2019-02-01T10:00:00 Nan values at the end
start_day = datetime.fromisoformat("2018-10-01T10:00:00")

start_date = start_day + index * relativedelta(months=+1)

In [95]:
number_of_folds = 15
from scipy import stats

In [101]:
number_of_folds = 15
NPw_o.get_folds(k = number_of_folds)

In [102]:
NPw_o.process_fold(12)

Finding best initial lr:   0%|          | 0/260 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 403it [00:00, ?it/s]

Predicting: 403it [00:00, ?it/s]

Predicting: 403it [00:00, ?it/s]

            0         49         98
0   21.139584  22.084459   22.27879
1   20.881529  21.818954   22.00943
2   20.551422  21.479319  21.664862
3   19.954285  20.864943  21.041565
4   19.154509   20.04208  20.206755
5   18.485937  19.354208  19.508892
6   17.999212  18.853432  19.000845
7   17.418055  18.255497  18.394228
8    16.70216  17.518936  17.646971
9   16.146547  16.947285  17.067019
10  15.858709  16.651136  16.766569
11  15.619327  16.404842  16.516701
12  15.311992  16.088636  16.195902
13  15.034834  15.803477  15.906601
14  14.788218  15.549743  15.649183
15  14.543778  15.298245  15.394033
16  14.611495  15.367918  15.464718
17   15.40269  16.181953  16.290573
18  16.789753  17.609058  17.738401
19  18.192146  19.051937  19.202232
20  19.302811  20.194664  20.361553
21  20.231533  21.150196  21.330961
22  20.964403  21.904221  22.095936
23  21.261984  22.210394  22.406553
24  21.136271  22.081051  22.275331
25  20.878113  21.815441  22.005865
26  20.547764  21.475555  21

In [97]:
for n_fold in range(number_of_folds):
    NPw_o.process_fold(n_fold)

Finding best initial lr:   0%|          | 0/260 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 403it [00:00, ?it/s]

Predicting: 403it [00:00, ?it/s]

Predicting: 403it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

Predicting: 371it [00:00, ?it/s]

Predicting: 371it [00:00, ?it/s]

Predicting: 371it [00:00, ?it/s]

ValueError: Length mismatch: Expected axis has 2 elements, new values have 3 elements

In [98]:
NPw_o.npw_df

,start_forecast,n_samples,offset_model,RMSE,MSE,MAE,actual_event,expected_event
strategic-molly,2017-04-05 11:00:00,26421,None,duth 7.063532 larm 7.338813 noa1 7.21...,duth 49.893491 larm 53.858170 noa1 51...,duth 5.440181 larm 5.437421 noa1 5.51...,4.0,None
protestant-leeanne,2017-07-11 22:00:00,26421,None,duth 7.994110 larm 8.061520 noa1 9.23...,duth 63.905794 larm 64.988100 noa1 85...,duth 7.553435 larm 7.504169 noa1 7.52...,24.0,None
normal-noemi,2017-10-17 09:00:00,26421,None,duth 8.111336 larm 8.984591 noa1 8.60...,duth 65.793773 larm 80.722878 noa1 73...,duth 6.537061 larm 7.147663 noa1 6.81...,-72.0,None
large-bobbee,2018-01-22 20:00:00,26421,None,duth 4.809079 larm 4.703161 noa1 4.43...,duth 23.127239 larm 22.119724 noa1 19...,duth 4.189451 larm 4.114289 noa1 3.87...,13.0,None
professional-mamie,2018-04-30 07:00:00,26421,None,duth 6.061431 larm 6.095970 noa1 5.81...,duth 36.740940 larm 37.160846 noa1 33...,duth 5.392784 larm 5.384164 noa1 5.14...,30.0,None
industrial-marion,2018-08-05 18:00:00,26421,None,duth 9.234764 larm 9.917687 noa1 9.03...,duth 85.280862 larm 98.360520 noa1 81...,duth 8.783921 larm 9.549890 noa1 8.67...,-953.0,None
growing-nananne,2018-11-11 05:00:00,26421,None,duth 8.208209 larm 8.972893 noa1 8.09...,duth 67.374700 larm 80.512809 noa1 65...,duth 6.875318 larm 7.552627 noa1 6.74...,3.0,None
quarrelsome-maureene,2019-02-16 16:00:00,26421,None,duth 3.634177 larm 3.126272 noa1 5.03...,duth 13.207246 larm 9.773579 noa1 25...,duth 3.191793 larm 2.702800 noa1 4.45...,10.0,None
conceptual-jewel,2019-05-25 03:00:00,26421,None,duth 2.934675 larm 4.837152 noa1 2.74...,duth 8.612318 larm 23.398039 noa1 7...,duth 2.457158 larm 3.186848 noa1 2.33...,-6.0,None
annual-guillema,2019-08-30 14:00:00,26421,None,duth 3.903591 larm 3.734786 noa1 3.54...,duth 15.238023 larm 13.948624 noa1 12...,duth 3.203086 larm 3.058964 noa1 2.90...,1.0,None


In [162]:
df_train.iloc[-(NPw_o.n_forecasts)]["ds"]

Timestamp('2017-05-09 18:00:00')

In [141]:
for index, (df_train, df_test) in enumerate(NPw_o.folds):
    if index == 10:
        print(df_test)


                      ds          y    ID
0    2019-02-19 18:00:00  19.734861  duth
1    2019-02-19 21:00:00  20.727333  duth
2    2019-02-20 00:00:00  20.738965  duth
3    2019-02-20 03:00:00  20.750598  duth
4    2019-02-20 06:00:00  20.762230  duth
...                  ...        ...   ...
4081 2019-08-08 09:00:00  24.958944  noa1
4082 2019-08-08 12:00:00  24.307972  noa1
4083 2019-08-08 15:00:00  24.940583  noa1
4084 2019-08-08 18:00:00  24.649556  noa1
4085 2019-08-08 21:00:00  19.739083  noa1

[4086 rows x 3 columns]


In [87]:
config_fc = ConfigForecast(
    start_forecast=start_date, offset_event=None
)
metrics= NPw_o.add_forecast(config_fc
)

2018-11-01 09:00:00
2018-10-30 09:00:00
                      ds          y    ID     depth       mag      dist  \
0    2017-11-01 12:00:00  25.725944  duth  0.000000  0.000000  0.000000   
1    2017-11-01 15:00:00  21.906417  duth  0.000000  0.000000  0.000000   
2    2017-11-01 18:00:00  20.057917  duth  0.000000  0.000000  0.000000   
3    2017-11-01 21:00:00  19.974917  duth  0.000000  0.000000  0.000000   
4    2017-11-02 00:00:00  20.154750  duth  0.000000  0.000000  0.000000   
...                  ...        ...   ...       ...       ...       ...   
9030 2017-10-11 21:00:00        NaN   NaN  0.025204  0.295455  0.002895   
9031 2017-10-14 09:00:00        NaN   NaN  0.021841  0.159091  0.019350   
9032 2017-10-23 00:00:00        NaN   NaN  0.031516  0.159091  0.032260   
9033 2017-10-24 09:00:00        NaN   NaN  0.021841  0.181818  0.035727   
9034 2017-10-29 06:00:00        NaN   NaN  0.049120  0.136364  0.028213   

       arc_cos   arc_sin        pr  
0     0.000000  0.0000

Finding best initial lr:   0%|          | 0/248 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 269it [00:00, ?it/s]

Predicting: 269it [00:00, ?it/s]

...

2018-11-01 09:00:00
      current                              pred                      
         duth       larm       noa1        duth       larm       noa1
0   25.530583  26.139306  25.153056  120.661316  30.584166  29.049374
1   24.821472  25.887722  25.218333  121.930656  30.873896  29.317024
2   21.021333  22.491556  21.754306    117.0709  29.764643  28.292316
3   20.926722  21.666611  21.131028  103.877068  26.753111  25.510319
4   19.224528  20.116389  19.245444   93.671204  24.423594  23.358356
5    7.181389   7.543333   7.291528   89.216438  23.406778  22.419044
6    7.835222   8.318778   8.560611   94.945381  24.714426  23.627022
7    9.835222  10.220639  10.398528  120.706322   30.59444  29.058865
8   11.149944  10.894250  11.092556   130.42453  32.812649  31.108006
9   11.607806  11.705278  12.284167   131.82518  33.132347  31.403339
10   7.529500   8.015222   8.493889  127.044052  32.041046  30.395214
11   7.153000   7.043500   7.642556  113.856667  29.030985  27.614578


Predicting: 269it [00:00, ?it/s]

Predicting: 269it [00:00, ?it/s]

Predicting: 269it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/carloscano/anaconda3/envs/neuralprophet/lib/python3.9/site-packages/neuralprophet/plot_model_parameters_plotly.py:373: RuntimeWarning:

invalid value encountered in divide




In [80]:
metrics[1]

FigureWidgetResampler({
    'data': [{'fill': 'none',
              'line': {'color': '#2d92ff', 'width': 2},
              'mode': 'lines',
              'name': '<b style="color:sandybrown">[R]</b> Trend <i style="color:#fc9944">~9h</i>',
              'type': 'scatter',
              'uid': '86e14430-d90e-4ed1-817a-91574773541c',
              'x': array([datetime.datetime(2017, 11, 1, 12, 0),
                          datetime.datetime(2017, 11, 1, 18, 0),
                          datetime.datetime(2017, 11, 1, 21, 0), ...,
                          datetime.datetime(2018, 10, 31, 18, 0),
                          datetime.datetime(2018, 10, 31, 21, 0),
                          datetime.datetime(2018, 11, 1, 6, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([46.43117191, 46.31756029, 46.26075448, ..., 65.7386202 , 65.7112081 ,
                          65.62897178]),
              'yaxis': 'y'},
             {'marker': {'color': '#2d92ff'},
             

In [ ]:
NPw_o.one_step_test(model, config_fc, df_test)

In [277]:
future = model.one_step(df_test, n_historic_predictions=24)

In [ ]:
forecast = model.predict(future, decompose=False, raw=True)
forecast_2  = forecast[forecast["ds"] == start_date].T

In [335]:
forecast_2.columns = df_test_2.columns  + "_pred"
df_test_2.columns =  df_test_2.columns  + "_current"

In [343]:
forecast_2 = forecast_2.drop(["ds" ,"ID"]).reset_index().drop("index", axis = 1)

In [414]:
df_test

,ds,y,ID,depth,mag,dist,arc_cos,arc_sin,pr
0,2019-03-27 10:00:00,24.289000,duth,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-03-27 11:00:00,24.899583,duth,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-03-27 12:00:00,23.751583,duth,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-03-27 13:00:00,23.437917,duth,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-03-27 14:00:00,23.762417,duth,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
427,2019-04-02 05:00:00,8.873250,noa1,0.0,0.0,0.0,0.0,0.0,0.0
428,2019-04-02 06:00:00,9.546583,noa1,0.0,0.0,0.0,0.0,0.0,0.0
429,2019-04-02 07:00:00,10.731167,noa1,0.0,0.0,0.0,0.0,0.0,0.0
430,2019-04-02 08:00:00,11.372333,noa1,0.0,0.0,0.0,0.0,0.0,0.0


In [416]:
pd.pivot(df_test, index="ds", columns="ID", values="y")[-24:]

ID,duth,larm,noa1
ds,,,
2019-04-01 10:00:00,24.909083,26.871500,27.418417
2019-04-01 11:00:00,24.880333,26.595500,26.971833
2019-04-01 12:00:00,25.879583,27.841750,28.300500
2019-04-01 13:00:00,24.773750,26.981000,26.955500
2019-04-01 14:00:00,23.035500,25.411333,24.747583
2019-04-01 15:00:00,21.813167,24.049583,23.176583
2019-04-01 16:00:00,21.707667,23.461250,22.821000
2019-04-01 17:00:00,21.727500,23.359750,22.887667
2019-04-01 18:00:00,21.717500,23.232750,23.120083


In [374]:
df_all = pd.concat([df_test_2.reset_index(), forecast_2], axis = 1).drop("ds", axis = 1)
df_all.columns = df_all.columns.str.split('_', expand=True)
df_all = df_all.swaplevel(axis=1) 
mse = df_all['current'].sub(df_all['pred']).pow(2).mean(axis=0)
mae = df_all['current'].sub(df_all['pred']).abs().mean(axis=0)
rmse = df_all['current'].sub(df_all['pred']).pow(2).mean(axis=0).pow(1/2)
results = pd.concat([mse, mae, rmse], axis = 1)
results

,0,1,2
duth,74.581824,6.296380,8.636077
larm,67.118254,6.268352,8.192573
noa1,51.606940,6.086715,7.183797


In [409]:
 pd.pivot(df_test, index="ds", columns="ID", values="y")[-(self.n_forecasts):]

KeyError: 'ds'

In [293]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix

df_test_2.apply(mean_squared_error, axis=0)


TypeError: mean_squared_error() missing 1 required positional argument: 'y_pred'